# My First GNN
I followed [Petar Veličković](https://www.youtube.com/channel/UC9bkKi8Us7yevvP1KIBQHog) [talk](https://www.youtube.com/watch?v=8owQBFAHw7E) on the TensorFlow Channel and implemented the code in PyTorch (originaly implemented in TensorFlow)

The task was to classifie papers catagory by using a word vector and a graph connecting papers to their citetions.

Dataset: [CORA](https://relational.fit.cvut.cz/dataset/CORA)

In [241]:
import pandas as pd
import numpy as np
import spektral
import torch
from torch import nn
from torch.optim import Adam

import time

In [89]:
dataset = spektral.datasets.citation.Citation('cora')
graph = dataset.graphs[0]
#   a: the adjacency matrix
#   x: the node features
#   e: the edge features
#   y: the labels


adj = graph.a
features = graph.x
e_features = graph.e
labels = torch.from_numpy(graph.y)
train_mask = torch.from_numpy(dataset.mask_tr)
val_mask = torch.from_numpy(dataset.mask_va)
test_mask = torch.from_numpy(dataset.mask_te)

adj = torch.from_numpy(adj + np.eye(adj.shape[0]))
features = torch.from_numpy(features).type(torch.float32)
adj = adj.type(torch.float32)

print(features.shape)
print(adj.shape)
print(labels.shape)

print("Train#:\t", torch.sum(train_mask))
print("Val#:\t", torch.sum(val_mask))
print("Test#:\t", torch.sum(test_mask))

torch.Size([2708, 1433])
torch.Size([2708, 2708])
torch.Size([2708, 7])
Train#:	 tensor(140)
Val#:	 tensor(500)
Test#:	 tensor(1000)


List of catagories:
- Case_Based
- Genetic_Algorithms
- Neural_Networks
- Probabilistic_Methods
- Reinforcement_Learning
- Rule_Learning
- Theory

In [246]:
def masked_softmax_cross_entropy(logits,labels,mask):
    loss = nn.CrossEntropyLoss(reduction='none')(logits,torch.argmax(labels,dim=1))
    mask = mask.type(torch.float32)
    mask /= torch.mean(mask)
    loss *= mask
    return torch.mean(loss)

def masked_accuracy(logits,labels, mask):
    correct_prediction = (torch.argmax(logits, 1) == torch.argmax(labels, 1))
    accuracy_all = correct_prediction.type(torch.float32)
    mask = mask.type(torch.float32)
    mask /= torch.mean(mask)
    accuracy_all *= mask
    
    return torch.mean(accuracy_all)

In [212]:
class GnnLayer(nn.Module):
    def __init__(self, in_dim, out_dim, activation):
        super(GnnLayer,self).__init__()
        self.fc = nn.Linear(in_dim, out_dim)
        self.act = activation
        
    def forward(self, input_x, adj):
        x = self.fc(input_x)
        x = torch.matmul(adj,x)
        x = self.act(x)
        return x
        
class CoraGnn(nn.Module):
    def __init__(self,feat_len,units,lbl_len):
        super(CoraGnn, self).__init__()
        self.lyr_1 = GnnLayer(feat_len, units, nn.ReLU())
        self.lyr_2 = GnnLayer(units, lbl_len, nn.Identity())
        
    def forward(self,input_x, adj):
        x = self.lyr_1(input_x, adj)
        x = self.lyr_2(x, adj)
        
        return x

In [213]:
def train_cora(fts,adj,units,epochs,lr):
    net = CoraGnn(len(fts[0]),units,len(labels[0]))
    optimizer = Adam(net.parameters(),lr=lr)
    
    best_accuracy = 0.0
    
    for ep in range(epochs+1):
        logits = net(fts,adj)
        loss = masked_softmax_cross_entropy(logits,labels,train_mask)
        
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            logits = net(fts,adj)
            val_accuracy = masked_accuracy(logits, labels,val_mask)
            test_accuracy = masked_accuracy(logits, labels,val_mask)
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print("Epoch: {} | Train Loss: {:.3f} | Val Acc: {:.3f} | Test Acc: {:.3f}".format(ep,loss.item(),val_accuracy.item(),test_accuracy.item()))
    print("Done!")

In [233]:
train_cora(features,adj,32,200,1e-2)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [215]:
train_cora(features,torch.eye(adj.shape[0]),32,200,1e-2)

Epoch: 0 | Train Loss: 1.949 | Val Acc: 0.240 | Test Acc: 0.240
Epoch: 1 | Train Loss: 1.835 | Val Acc: 0.314 | Test Acc: 0.314
Epoch: 2 | Train Loss: 1.705 | Val Acc: 0.360 | Test Acc: 0.360
Epoch: 3 | Train Loss: 1.538 | Val Acc: 0.394 | Test Acc: 0.394
Epoch: 4 | Train Loss: 1.346 | Val Acc: 0.434 | Test Acc: 0.434
Epoch: 5 | Train Loss: 1.138 | Val Acc: 0.444 | Test Acc: 0.444
Epoch: 6 | Train Loss: 0.929 | Val Acc: 0.454 | Test Acc: 0.454
Epoch: 7 | Train Loss: 0.732 | Val Acc: 0.468 | Test Acc: 0.468
Epoch: 8 | Train Loss: 0.560 | Val Acc: 0.470 | Test Acc: 0.470
Epoch: 9 | Train Loss: 0.418 | Val Acc: 0.478 | Test Acc: 0.478
Epoch: 10 | Train Loss: 0.309 | Val Acc: 0.488 | Test Acc: 0.488
Epoch: 11 | Train Loss: 0.229 | Val Acc: 0.494 | Test Acc: 0.494
Epoch: 12 | Train Loss: 0.171 | Val Acc: 0.498 | Test Acc: 0.498
Epoch: 13 | Train Loss: 0.130 | Val Acc: 0.500 | Test Acc: 0.500
Epoch: 14 | Train Loss: 0.100 | Val Acc: 0.512 | Test Acc: 0.512
Epoch: 15 | Train Loss: 0.078 | Val

In [216]:
deg = torch.sum(adj, dim=1)
train_cora(features,adj/deg,32,300,1e-2)

Epoch: 0 | Train Loss: 1.951 | Val Acc: 0.558 | Test Acc: 0.558
Epoch: 1 | Train Loss: 1.866 | Val Acc: 0.562 | Test Acc: 0.562
Epoch: 3 | Train Loss: 1.626 | Val Acc: 0.564 | Test Acc: 0.564
Epoch: 4 | Train Loss: 1.483 | Val Acc: 0.598 | Test Acc: 0.598
Epoch: 5 | Train Loss: 1.333 | Val Acc: 0.630 | Test Acc: 0.630
Epoch: 6 | Train Loss: 1.181 | Val Acc: 0.674 | Test Acc: 0.674
Epoch: 7 | Train Loss: 1.033 | Val Acc: 0.706 | Test Acc: 0.706
Epoch: 8 | Train Loss: 0.890 | Val Acc: 0.722 | Test Acc: 0.722
Epoch: 9 | Train Loss: 0.759 | Val Acc: 0.736 | Test Acc: 0.736
Epoch: 15 | Train Loss: 0.268 | Val Acc: 0.748 | Test Acc: 0.748
Epoch: 16 | Train Loss: 0.215 | Val Acc: 0.758 | Test Acc: 0.758
Epoch: 17 | Train Loss: 0.169 | Val Acc: 0.770 | Test Acc: 0.770
Epoch: 18 | Train Loss: 0.130 | Val Acc: 0.774 | Test Acc: 0.774
Done!


In [217]:
norm_deg = torch.diag(1.0/torch.sqrt(deg))
norm_adj = torch.matmul(norm_deg,torch.matmul(adj,norm_deg))

train_cora(features,norm_adj,32,200,1e-2)

Epoch: 0 | Train Loss: 1.950 | Val Acc: 0.126 | Test Acc: 0.126
Epoch: 1 | Train Loss: 1.861 | Val Acc: 0.424 | Test Acc: 0.424
Epoch: 2 | Train Loss: 1.745 | Val Acc: 0.596 | Test Acc: 0.596
Epoch: 3 | Train Loss: 1.608 | Val Acc: 0.670 | Test Acc: 0.670
Epoch: 4 | Train Loss: 1.458 | Val Acc: 0.720 | Test Acc: 0.720
Epoch: 5 | Train Loss: 1.303 | Val Acc: 0.744 | Test Acc: 0.744
Epoch: 6 | Train Loss: 1.146 | Val Acc: 0.748 | Test Acc: 0.748
Epoch: 7 | Train Loss: 0.993 | Val Acc: 0.758 | Test Acc: 0.758
Epoch: 8 | Train Loss: 0.846 | Val Acc: 0.778 | Test Acc: 0.778
Epoch: 9 | Train Loss: 0.708 | Val Acc: 0.790 | Test Acc: 0.790
Done!
